# OCR (_Optical Character Recognition_)

## Objective:

    - The main objective of this project is to extract the customer information from the monthly rechnung (bill).
    - Initially the "Pdf" is converted into the "Image" and then 'opencv' library is used for adjusting the color and thershold.
        + While converting the pdf to image. download and mention the 'Poppler' location
    - In this 'numpy' library is used for extracting the desired portion for the 'image' for the futher process
    - 'Tesseract' library is used for converting the image to string and data
        + download and mention the tesseract location
    - 'Regex' is utilized here for extracting the useful information like all details.
    - Then a dataframe is created for storing all details and further dataframe is stored as ".csv" file.
    - New Database connection is established for storing the .csv files and new tables are created for storing the image files
    - In this project postgresql database is used.


### 1. Importing and installing all libraries

In [1]:
%pip install pdf2image
%pip install tesseract

In [2]:
import os
import cv2 as cv;
import pandas as pd;
import numpy as np;
import pytesseract;
import re;
from PIL import Image

### 2. Converting the pdf to image

In [3]:
from pdf2image import convert_from_path
images = convert_from_path('Sample_test.pdf', poppler_path= r'E:\integrify\Notebooks\Pytorch\poppler\Release-21.11.0-0\poppler-21.11.0\Library\bin')

In [4]:
len(images)

2

### 3. Storing the images in the new folder

In [5]:
Label = 'test_samples'
if not os.path.exists(Label):
    os.mkdir(Label)

In [6]:
os.chdir('test_samples')
for i, image in enumerate(images):
    test_cases = 'image'+str(i)+'.jpg'
    image.save(test_cases, "JPEG")

In [7]:
img = cv.imread('image0.jpg')
img = cv.resize(img, (1654,2339)) 

In [8]:
cv.imshow("Image", img)
cv.waitKey(0) # K -capital
cv.destroyAllWindows()

### 4. Converting into the image into numpy array

In [9]:
image_array = np.array(img)

In [10]:
len(image_array)

2339

In [11]:
check = Image.fromarray(image_array[1200:1400], 'RGB')
check.show()

### 5. Converting image into string

In [12]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(check)
print(text)

Mobilfunk-Vertragsabrechnungen

 

Mobilfunk-Rufnummer 0176 / 47344855 Vertragsnummer: 224983810
Mobilfunknetz: Telekom Tarif: green LTE 6 GB
Teilnehmer: Nikhil Raj Mule

 

 

 



### 5. Preprocessing for extracting the customer information

In [13]:
final_text = text.replace('\n','')

In [14]:
final_text

'Mobilfunk-Vertragsabrechnungen Mobilfunk-Rufnummer 0176 / 47344855 Vertragsnummer: 224983810Mobilfunknetz: Telekom Tarif: green LTE 6 GBTeilnehmer: Nikhil Raj Mule   '

In [15]:
data_ = re.findall(r"(?<=nummer\s)\d+\s\/\s\d+|(?<=nummer\:\s)\d+|(?<=netz\:\s)\w+|(?<=rif\:\s)\w+\s\w+\s\w+\s\wB|(?<=nehmer\:\s)\w+\s\w+\s\w+", final_text)
data_

['0176 / 47344855',
 '224983810',
 'Telekom',
 'green LTE 6 GB',
 'Nikhil Raj Mule']

### 6. Creating and saving the dataframe as .CSV file

In [16]:
phone_bill_data = pd.DataFrame(data= {'Handy_nummer': [data_[0]], 'Vertragsnummer': [data_[1]], 
                    'Mobilfunknetz': [data_[2]], 'Tarif': [data_[3]], 'Teilnehmer': [data_[4]]})
phone_bill_data

,Handy_nummer,Vertragsnummer,Mobilfunknetz,Tarif,Teilnehmer
0,0176 / 47344855,224983810,Telekom,green LTE 6 GB,Nikhil Raj Mule


In [17]:
phone_bill_data.to_csv('bill_data.csv')

### 7. Connecting to the database for storing the .csv fileto the database

In [19]:
# hiding the password
from getpass import getpass

In [18]:
param_dict={
    "host":"localhost",
    "database":"phone_bills",
    "user":"***",
    "password": "****"
}

In [19]:
%pip install psycopg2

In [20]:
from sqlalchemy.engine import create_engine
import psycopg2

In [21]:
connect = "postgresql://%s:%s@%s:5432/%s" %(param_dict["user"], param_dict["password"], param_dict["host"], param_dict["database"])

In [22]:
engine = create_engine(connect)
cur = engine.connect()

### 8. Saving the .csv file to the new database

In [24]:
phone_bill_data.to_sql("Phone_bills_1", cur)

### 9. Saving the image into the database

In [25]:
import psycopg2

# Connect to your postgres DB
conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
curr = conn.cursor()

# Execute a query
curr.execute('select * from "Phone_bills_1"')

# Retrieve query results
curr.fetchall()

[(0,
  '0176 / 47344855',
  '224983810',
  'Telekom',
  'green LTE 6 GB',
  'Nikhil Raj Mule')]

### 10. Creating the new table for storing the images

In [26]:
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """ CREATE TABLE image_data (
                serial_number SERIAL PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                images BYTEA NOT NULL)
        """)
    conn = None
    try:
        # read the connection parameters
        # connect to the PostgreSQL server
        conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
        cur = conn.cursor()
        # create table one by one
        cur.execute(commands)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

if __name__ == '__main__':
    create_tables()

### 11. Finally adding the image to the table

In [27]:
def write_blob(serial_number, path_to_file, file_extension):
    """ insert a BLOB into a table """
    conn = None
    try:
        # read data from a picture
        drawing = open(path_to_file,'rb').read()
        # read database configuration
        conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
        # connect to the PostgresQL database
        # create a new cursor object
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute("INSERT INTO image_data(serial_number,file_extension, images)" +
                    "VALUES(%s,%s,%s)",
                    (serial_number, file_extension, psycopg2.Binary(drawing)))
        # commit the changes to the database
        conn.commit()
        # close the communication with the PostgresQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [28]:
write_blob(1,'image0.jpg', 'jpg')

### 12. Checking the image loaction

In [29]:
def read_blob(path_to_dir):
    """ read BLOB data from a table """
    conn = None
    try:
        # read database configuration
        # connect to the PostgresQL database
        conn = psycopg2.connect("dbname=phone_bills user=*** password=**** host=localhost")
        # connect to the PostgresQL database
        # create a new cursor object
        cur = conn.cursor()
        # execute the SELECT statement
        cur.execute(""" SELECT *
                        FROM image_data
                        """)

        blob = cur.fetchone()
        print(blob)
        open(path_to_dir +'.' + blob[1], 'wb').write(blob[2])
        # close the communication with the PostgresQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [30]:
read_blob("image0")

(1, 'jpg', <memory at 0x00000222AEC3AF40>)
